In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import os

%matplotlib inline

In [ ]:
parent_dir = os.path.split(os.getcwd())[0]

In [ ]:
odin = pd.read_csv(parent_dir + '\\Data\\New\\Ovin_final.csv',  index_col=0)
zones = gpd.read_file(parent_dir + '\\Data\\New\\lms_zone_du_new.shp') # LMS Zone data

Create dataframe to store results

In [ ]:
tb_data = zones[['ZONE_ID']]

## Modal Split

For modal split, do not include "moped" or "other" in the modal split percentages. Calculate modal split for both origin and destination zone

In [ ]:
odin_modal = odin[(~odin.KHvm.isin([5, 8])) & (~odin.KHvm.isnull())]

In [ ]:
zone_type = ['VertZone', 'AankZone', 'WoonZone']
labels_vert = [['Car_driver_o', 'Car_passenger_o', 'Train_o', 'BTM_o', 'Bike_o', 'Walk_o'],
                ['Car_driver_d', 'Car_passenger_d', 'Train_d', 'BTM_d', 'Bike_d', 'Walk_d'],
                ['Car_driver_w', 'Car_passenger_w', 'Train_w', 'BTM_w', 'Bike_w', 'Walk_w']]

In [ ]:
aank_tot = odin_modal[['AankZone', 'FactorV_final']].groupby('AankZone').sum()
vert_tot = odin_modal[['VertZone', 'FactorV_final']].groupby('VertZone').sum()
woon_tot = odin_modal[['WoonZone', 'FactorV_final']].groupby('WoonZone').sum()


tot_list = [vert_tot, aank_tot, woon_tot]

In [ ]:
m_list = [1, 2, 3, 4, 6, 7]

for i in range(3):

    for m in range(len(m_list)):

        split = odin_modal[odin_modal.KHvm == m_list[m]][[zone_type[i], 'FactorV_final']].groupby(zone_type[i]).sum()
    	
        split = np.round(split / tot_list[i] * 100, 2)

        tb_data = tb_data.merge(split, left_on='ZONE_ID', right_index=True, how='left')
        tb_data = tb_data.rename(columns={'FactorV_final': labels_vert[i][m]})


In [ ]:
tb_data

## Travel time, distance and frequency

In [ ]:
def f_op(x):
    indices = ~np.isnan(x)
    # print(indices.any())
    if indices.any():
        return np.average(x[indices], weights=odin[~odin.OPID.duplicated()].loc[x.index[indices], 'FactorV_final'])
    else:
        return np.nan

def f(x):
    indices = ~np.isnan(x)
    # print(indices.any())
    if indices.any():
        return np.average(x[indices], weights=odin.loc[x.index[indices], 'FactorV_final'])
    else:
        return np.nan

Calculate both the average time/ distance of trips and of the day (the latter is based on home zone)

In [ ]:
tt_o = np.round(odin.groupby(['VertZone']).agg(Travel_time_o=('Reisduur', f)), 2)
tt_d = np.round(odin.groupby(['AankZone']).agg(Travel_time_d=('Reisduur', f)), 2)
td_o = np.round(odin.groupby(['VertZone']).agg(Distance_o=('AfstV', f)) / 10, 2)
td_d = np.round(odin.groupby(['AankZone']).agg(Distance_d=('AfstV', f)) / 10, 2)



tb_data = tb_data.merge(tt_o, left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.merge(tt_d, left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.merge(td_o, left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.merge(td_d, left_on='ZONE_ID', right_index=True, how='left')


In [ ]:
tb_data

Determine total travel time, dist and frequency over the day based on home zone

In [ ]:
tt_w = odin[~odin.OPID.duplicated()].groupby(['WoonZone']).agg(Travel_time_w=('ReisduurOP', f_op))
td_w = odin[~odin.OPID.duplicated()].groupby(['WoonZone']).agg(Distance_w=('AfstandOP', f_op))
freq_w = odin[~odin.OPID.duplicated()].groupby(['WoonZone']).agg(Freq_w=('AantVpl', f_op))


In [ ]:
tb_data = tb_data.merge(np.round(tt_w, 2), left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.merge(np.round(td_w / 10, 2), left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.merge(np.round(freq_w, 2), left_on='ZONE_ID', right_index=True, how='left')


In [ ]:
tb_data

## Count number of datapoints for each zone

In [ ]:
Count_o = odin_modal.VertZone.value_counts()
Count_d = odin_modal.AankZone.value_counts()
Count_w = odin_modal.WoonZone.value_counts()


In [ ]:
tb_data = tb_data.merge(Count_o, left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.rename(columns={'count':'Count_o'})
tb_data = tb_data.merge(Count_d, left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.rename(columns={'count':'Count_d'})
tb_data = tb_data.merge(Count_w, left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.rename(columns={'count':'Count_w'})



In [ ]:
tb_data

In [ ]:
tb_data.columns

## Sum FactorV_final

In [ ]:
factor_o = odin_modal[['VertZone', 'FactorV_final']].groupby('VertZone').sum()
factor_d = odin_modal[['AankZone', 'FactorV_final']].groupby('AankZone').sum()
factor_w = odin_modal[['WoonZone', 'FactorV_final']].groupby('WoonZone').sum()


In [ ]:
tb_data = tb_data.merge(factor_o, left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.rename(columns={'FactorV_final':'Factor_o'})
tb_data = tb_data.merge(factor_d, left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.rename(columns={'FactorV_final':'Factor_d'})
tb_data = tb_data.merge(factor_w, left_on='ZONE_ID', right_index=True, how='left')
tb_data = tb_data.rename(columns={'FactorV_final':'Factor_w'})

In [ ]:
tb_data

## Save to csv

In [ ]:
tb_data.to_csv(parent_dir + '\\Data\\New\\lms_zone_ovin_travel_behaviour_newF.csv')